In [38]:
import requests
import json
import pandas as pd
import os

data_center_tag = pd.read_csv("Datacenters_for_tags_in_measurement.csv", keep_default_na=False)
print(data_center_tag)

def create_ping_json(datacenter_domain_name, probe_set):
        
    # I want to create several tags in the measurement
    #including data_center_name I created, Continent, provider name
    row = data_center_tag[data_center_tag["AU"] == datacenter_domain_name]
    dc_name = row["Datacenter_name"].iloc[0]
    dc_continet = row["Continent"].iloc[0]
    dc_provider = row["Provider"].iloc[0]
    if dc_provider == "Amazon EC2":
        dc_provider = "Amazon"
    
    probe_all_count = len(probe_set)
    
    ###### PARAMETER HERE #############
    # set here how many measurement can there be in one measurement
    limit_measurement = 10
    ###### PARAMETER END #############
    
    
    batch = -(probe_all_count // -limit_measurement) #ceiling, a simple approach not using math 
    
    for i in range(batch):
        
        start = i*25
        end = min((i+1)*25-1 , probe_all_count-1)
        probe_set_thisbatch = probe_set[start:end+1]
        
        probe_name_thisbatch = ",".join(probe_set_thisbatch)
        probe_count_thisbatch = len(probe_set_thisbatch)
        
        description_batch = "Ping-" + dc_name + "-" + dc_provider + "-" + str(i+1) + "of" + str(batch) 
        path_name = "./ping_json/" + description_batch + ".json"
        
        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json"
        }

        data = {
            "definitions": [
                {
                    "target": datacenter_domain_name,
                    "af": 4,
                    "packets": 3,
                    "size": 48,
                    "description": description_batch,
                    "interval": 3600,
                    "tags": ["ping", dc_name, dc_continet, dc_provider],
                    "resolve_on_probe": False,
                    "skip_dns_check": False,
                    "include_probe_id": False,
                    "type": "ping"
                }
            ],
            "probes": [
                {
                    "value": probe_name_thisbatch,
                    "type": "probes",
                    "requested": probe_count_thisbatch
                }
            ],
            "is_oneoff": False,
            "bill_to": "zhihang.wei@tum.de",
            "start_time": 1675818051,
            "stop_time": 1676426451
        }

        url = "https://atlas.ripe.net/api/v2/measurements/?key=YOUR_KEY_HERE"

        response = requests.post(url, headers=headers, json=data)

        # write the response to a JSON file
        
        directory = os.path.dirname(path_name)

        if not os.path.exists(directory):
            os.makedirs(directory)
        
        with open(path_name, "w") as file:
            json.dump(response.json(), file)
    

    
    






    index                                            AU Continent  \
0       1           ap-southeast-2.ec2.cloudharmony.net        OC   
1       2         australia-east.azure.cloudharmony.net        OC   
2       3     australia-southeast1.gce.cloudharmony.net        OC   
3       4           brazil-south.azure.cloudharmony.net        SA   
4       5                sa-east-1.ec2.cloudharmony.net        SA   
5       6       southamerica-east1.gce.cloudharmony.net        SA   
6       7             ca-central-1.ec2.cloudharmony.net        NA   
7       8            canada-east.azure.cloudharmony.net        NA   
8       9  northamerica-northeast1.gce.cloudharmony.net        NA   
9      10             eu-central-1.ec2.cloudharmony.net        EU   
10     11             europe-west3.gce.cloudharmony.net        EU   
11     12    germany-westcentral.azure.cloudharmony.net        EU   
12     13                eu-west-2.ec2.cloudharmony.net        EU   
13     14             europe-west2

In [39]:
test_probe_set = [str(i + 1000) for i in range(28)]
print(test_probe_set)
test_domain = "ap-southeast-2.ec2.cloudharmony.net"

['1000', '1001', '1002', '1003', '1004', '1005', '1006', '1007', '1008', '1009', '1010', '1011', '1012', '1013', '1014', '1015', '1016', '1017', '1018', '1019', '1020', '1021', '1022', '1023', '1024', '1025', '1026', '1027']


In [41]:
create_ping_json(test_domain, test_probe_set)

In [37]:
dc_provider = r"Amazon EC2"
if dc_provider == "Amazon EC2":
    dc_provider == "Amazon"
print(dc_provider)

Amazon EC2
